In [ ]:
#Based on https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/convolutional_network.py:
#
#A Convolutional Network implementation example using TensorFlow library.
#This example is using the MNIST database of handwritten digits
#(http://yann.lecun.com/exdb/mnist/)
#
#Author: Aymeric Damien
#Project: https://github.com/aymericdamien/TensorFlow-Examples/
#
#
#*****************************************************************
#
#Licensed Materials - Property of IBM
#
#(C) Copyright IBM Corp. 2017, 2019. All Rights Reserved.
#
#US Government Users Restricted Rights - Use, duplication or
#disclosure restricted by GSA ADP Schedule Contract with IBM Corp.
#
#*****************************************************************


# Distributed Deep Learning

IBM Watson Machine Learning (ML) Community Edition feature Distributed Deep Learning (or DDL) utilizes a MPI-based communication library which is specifically optimized for deep learning training.  An application integrated with DDL becomes an MPI-application, which will allow the use of the <i>ddlrun</i> command to invoke the job in parallel across a cluster of systems.  DDL understands multi-tier network environments+ and uses different libraries (for example NCCL) and algorithms to get the best performance in multi-node, multi-GPU environments.

IBM Watson ML Community Edition Distributed Deep Learning has been integrated with the IBM Caffe, Pytorch and TensorFlow packages. 

As part of Watson ML Community Edition, there are a few requirements to utilize DDL:

1. The same version of Watson ML Community Edition installed on the nodes

In [ ]:
!conda list "powerai|ddl|mpi"

2. The DDL master node must be able to login to the other nodes via non-prompted SSH
3. Linux system firewalls might need to be adjusted to pass MPI traffic. This adjustment might be done broadly as shown below .  The first command has already been run.

In [ ]:
#sudo firewall-cmd --zone=public --add-port=1024-65535/tcp
!sudo firewall-cmd --zone=public --list-ports

Additional details on IBM Watson ML Community Edition's [Distributed Deep Learning](https://www.ibm.com/support/knowledgecenter/en/SS5SF7_1.6.1/navigation/wmlce_getstarted_ddl.html), including other Tutorials, can be found in the Knowledge Center link provided here.

## Lab Environment


In our lab you are using an IBM POWER9-based [Power System AC922](https://www.ibm.com/us-en/marketplace/power-systems-ac922) with four NVIDIA Volta V100 GPUs attached via NVLINK 2.0.
![AC922 with NVLink](https://developer.ibm.com/linuxonpower/wp-content/uploads/sites/105/2018/12/tflms.png)

The NVLINK high-speed bus technology is available on x86 computers, but <b>only</b> for GPU to GPU connections.  IBM's POWER9 CPU has NVLINK 2.0 connections directly on the CPU to allow for high-speed bandwidth from the GPU to system memory.

In [ ]:
# View the GPU devices, NVIDIA Driver version, NVIDIA CUDA version and more.  You can see these are 16GB GPUs
!/usr/bin/nvidia-smi

Now because we're sharing this system among 4 students, we have to play nice and specify which single GPU we'll be using.

| <b>Team | GPU</b> |
| ---- | --- |
| team01, team05, team09, team13, team17 | 0 |
| team02, team06, team10, team14, team18 | 1 |
| team03, team07, team11, team15, team19 | 2 |
| team04, team08, team12, team16, team20 | 3 |

We can check if the <code>CUDA_VISIBLE_DEVICES</code> environment variable is set, and if not, we need to set it.


In [ ]:
import os
print(os.environ['CUDA_VISIBLE_DEVICES'])
## If this doesn't return something, then edit the following line with the proper GPU number
## Then uncomment the line an re-run this cell
#%env CUDA_VISIBLE_DEVICES=0

### MNIST

The [MNIST](http://yann.lecun.com/exdb/mnist/index.html) model and dataset is a relatively simple image classification neural network.

To get an idea of timing, let's first see an unmodified run.  This should take less than one minute to complete.

In [ ]:
!time python ./tf/mnist.py

Your numbers may vary, but here is an example ending to the basic run that took approximately 47 seconds.
><code>...
Iter 19700, Minibatch Loss= 0.000095, Training Accuracy= 1.00000
Iter 19800, Minibatch Loss= 0.013908, Training Accuracy= 1.00000
Iter 19900, Minibatch Loss= 0.000127, Training Accuracy= 1.00000
Testing Accuracy: 0.98828125
real	0m47.010s
user	0m55.445s
sys     0m7.983s
</code>

So now we have a starting point.

<div class="alert alert-block alert-info">
    <b> IMPORTANT NOTE! </b>
    
For the remainder of the lab, we're going to be using more than one GPU (either on your assigned host, or across more than one host).  However, since we only have 5 hosts, with 4 GPUs per host, that means we have only 20 GPUs.  It is highly likely that collisions will occur (GPUs are already in use by another process, aka, your other lab members).  Be patient and re-run the cell again.

This is effectively a command-line lab and as such, these collisions will occur.

For more effective use of GPU resources on one or more systems, there are products like __[IBM Watson Machine Learning Accelerator](https://www.ibm.com/us-en/marketplace/deep-learning-platform)__ and __[IBM Spectrum LSF](https://www.ibm.com/us-en/marketplace/hpc-workload-management)__ that can be used to schedule GPU workloads effectively without any collisions.
</div>

### DDL with one GPU

Let's start with a single GPU using the DDL run command, <code>ddlrun</code>.

We're going to pass a parameter to the underlying __[IBM Spectrum MPI](https://www.ibm.com/us-en/marketplace/spectrum-mpi)__ execution engine (mpirun) telling it we do NOT have InfiniBand (**--mpiarg -pami_noib**)

We're also going to utilize only ONE GPU (**--accelerators 1**) at this time.

In [ ]:
!time ddlrun --mpiarg -pami_noib --accelerators 1 python $PWD/tf/mnist-env.py

Looking at the larger output this command generated you can see details on some of the extra work required in the DDL setup, but in the end it looks very similar:<p>
<code>...
DDL 0] Iter 19800, Minibatch Loss= 0.139387, Training Accuracy= 0.96000
DDL 0] Iter 19900, Minibatch Loss= 0.195699, Training Accuracy= 0.95000
DDL 0] Optimization Finished!
DDL 0] Testing Accuracy: 0.9
2019-04-01 04:55:50.880779: I ddl_MDR_ops.cc:348] [MPI:0   ] calling ddl_finalize

real	0m14.991s
user	0m18.061s
sys     0m5.267s
</code>

But with a faster time to completion!

### DDL with >1 GPU on a single host

We've got more than one GPU in the host, so let's try and use two GPUs.

In [ ]:
!time ddlrun --mpiarg -pami_noib --accelerators 2 python $PWD/tf/mnist-env.py

<b> Uh-oh! </b> What happened?

The key is in the message here:
<code>tensorflow.python.framework.errors_impl.InvalidArgumentError: 'visible_device_list' listed an invalid GPU id '1' but visible device count is 1</code>

Remember above where we checked our environment variable for a single GPU assignment?
CUDA_VISIBLE_DEVICES

We need to blank out that variable and try again.

In [ ]:
import os
del os.environ['CUDA_VISIBLE_DEVICES']

In [ ]:
!time ddlrun --mpiarg -pami_noib --accelerators 2 python $PWD/tf/mnist-env.py

There's a difference in the 2 GPU output and the 1 GPU output:<p>
<code>...
<b>DDL 0]</b> Iter 19700, Minibatch Loss= 0.196856, Training Accuracy= 0.92000
DDL 0] Iter 19800, Minibatch Loss= 0.107701, Training Accuracy= 0.96000
<b>DDL 1]</b> Iter 19800, Minibatch Loss= 0.055850, Training Accuracy= 0.99000
DDL 1] Iter 19900, Minibatch Loss= 0.091027, Training Accuracy= 0.98000
DDL 1] Optimization Finished!
DDL 0] Iter 19900, Minibatch Loss= 0.148139, Training Accuracy= 0.93000
DDL 0] Optimization Finished!

DDL 1] Testing Accuracy: 0.89
DDL 0] Testing Accuracy: 0.93
2019-04-01 05:14:33.736428: I ddl_MDR_ops.cc:348] [MPI:1   ] calling ddl_finalize
2019-04-01 05:14:33.741907: I ddl_MDR_ops.cc:348] [MPI:0   ] calling ddl_finalize


real	0m15.050s
user	0m30.293s
sys	0m8.919s
</code>

What you're seeing here are the two distributed DL processes spawned by DDL.

### DDL across more than one host

Okay, now we need to determine the hosts to run our multi-node DDL training job.  We're using the "-10" hostname to refer to the higher-speed Ethernet (which was recently changed to 40GbE, but the hostnames were not changed) versus the regular 1GbE.

| <b>Team | | NODE pair</b> |
| ---- | | --- |
| team01 through team04 | | ac922a-10, ac922b-10 |
| team05 through team08 | | ac922b-10, ac922c-10 |
| team09 through team12 | | ac922c-10, ac922d-10 |
| team13 through team16 | | ac922d-10, ac922e-10 |
| team17 through team20 | | ac922e-10, ac922a-10 |

Type (or copy and paste) the host pair contents into the ddlrun command host specification replacing **NODE1,NODE2**

Now we're going to drop to using 1 GPU <b>per host</b> and since we're spanning hosts, we replace the "--mpiarg -pami_noib" with "--tcp" to specify we're not using the lower-level verbs protocol over InfiniBand, but just regular IP communication.  In addition, we're passing an environment variable down to the NCCL layer to specify NOT to use the incorrect interface (our lab environment does not have the same 10 GigE interface name across the nodes so we can't just specify one interface name).

In [ ]:
!time ddlrun -H NODE1,NODE2 --tcp --accelerators 1 \
   --mpiarg "-x NCCL_SOCKET_IFNAME=^veth,docker,lo" python $PWD/tf/mnist-env.py

Hmm, the time to complete should be equal to, or greater than the single-node run; but less than the first (non-DDL) run.

There's a couple of reasons for that:
* We are using Ethernet after all, not InfiniBand, so the communications protocol overhead is higher
* There is set up and discovery of the hosts and the topology.  If we knew exactly the set up ahead of time, we can specify the <code>--rankfile fn</code> option to provide those details.
* Network topologies are also checked, ssh verified, etc.

We can skip many of these checks by including the <code>--skipchecks</code> flag which will produce this warning as the first line of the output:

[WARN DDL-2-28] The SKIPCHECKS flag is set. This removes the ability for 'ddlrun' to detect remote hardware configurations and can cause errors in non-homogenous cluster environments - i.e., machines with varying core or GPU counts.
Please see /tmp/DDLRUN/DDLRUN.XrW9sLjXM8NE/ddlrun.log for detailed log.


Removing the checks should shave a few seconds off the overall run time.

What about if we double the number of iterations?

In [ ]:
!time ddlrun -H NODE1,NODE2 --tcp --accelerators 1 --skipchecks \
      -mpiarg "-x NCCL_SOCKET_IFNAME=^veth,docker,lo" python $PWD/tf/mnist-env.py --num_iterations 40000

#### Conclusion

Hopefully you saw that doubling the number of iterations in a distributed job did not double the overall run time!  In fact, it may have completed the job in roughly the same amount of time or a small additional delta.  For fun, try ten times the number of iterations (200,000) and see if it takes ten times the amount of time!